#**ESMFold_advanced**
for more details see: [Github](https://github.com/facebookresearch/esm/tree/main/esm), [Preprint](https://www.biorxiv.org/content/10.1101/2022.07.20.500902v1)

#### **Tips and Instructions**
- click the little ▶ play icon to the left of each cell below.

#### **Colab Limitations**
- On Tesla T4 (typical free colab GPU), max total length ~ 900



In [1]:
%%time
#@title install
#@markdown install ESMFold, OpenFold and download Params (~2min 30s)

import os, time
if not os.path.isfile("esmfold.model"):
  # download esmfold params
  os.system("apt-get install aria2 -qq")
  os.system("aria2c -q -x 16 https://colabfold.steineggerlab.workers.dev/esm/esmfold.model &")

  # install libs
  print("installing libs...")
  os.system("pip install -q omegaconf pytorch_lightning biopython ml_collections einops py3Dmol modelcif")
  os.system("pip install -q git+https://github.com/NVIDIA/dllogger.git")

  print("installing openfold...")
  # install openfold
  os.system(f"pip install -q git+https://github.com/sokrypton/openfold.git")

  print("installing esmfold...")
  # install esmfold
  os.system(f"pip install -q git+https://github.com/sokrypton/esm.git@beta")

  # wait for Params to finish downloading...
  if not os.path.isfile("esmfold.model"):
    # backup source!
    os.system("aria2c -q -x 16 https://files.ipd.uw.edu/pub/esmfold/esmfold.model")
  else:
    while os.path.isfile("esmfold.model.aria2"):
      time.sleep(5)

CPU times: user 34 µs, sys: 0 ns, total: 34 µs
Wall time: 31.9 µs


In [2]:
%%time
import torch
from torch.utils.data import Dataset, DataLoader
from string import ascii_uppercase, ascii_lowercase
import hashlib, re, os
import numpy as np
from jax.tree_util import tree_map
import matplotlib.pyplot as plt
from scipy.special import softmax
import pandas as pd
from google.colab import files



def parse_output(output):
  pae = (output["aligned_confidence_probs"][0] * np.arange(64)).mean(-1) * 31
  plddt = output["plddt"][0,:,1]

  bins = np.append(0,np.linspace(2.3125,21.6875,63))
  sm_contacts = softmax(output["distogram_logits"],-1)[0]
  sm_contacts = sm_contacts[...,bins<8].sum(-1)
  xyz = output["positions"][-1,0,:,1]
  mask = output["atom37_atom_exists"][0,:,1] == 1
  o = {"pae":pae[mask,:][:,mask],
       "plddt":plddt[mask],
       "sm_contacts":sm_contacts[mask,:][:,mask],
       "xyz":xyz[mask]}
  if "contacts" in output["lm_output"]:
    lm_contacts = output["lm_output"]["contacts"].astype(float)[0]
    o["lm_contacts"] = lm_contacts[mask,:][:,mask]
  return o

def get_hash(x): return hashlib.sha1(x.encode()).hexdigest()
alphabet_list = list(ascii_uppercase+ascii_lowercase)






def esmfold_plddt(model, sequences, num_recycles=0, chain_linker=2, masking_rate=0.5, get_LM_contacts=True, samples=8):
    plddt_ls = []
    stochastic_mode = "LM"

    with torch.no_grad():
        for seed in range(samples):
            if samples is None:
                seed = "default"
                mask_rate = 0.0
                model.train(False)
            else:
                torch.manual_seed(seed)
                mask_rate = masking_rate if "LM" in stochastic_mode else 0.0
                model.train("SM" in stochastic_mode)

            output = model.infer(sequences,
                                num_recycles=num_recycles,
                                chain_linker="X" * chain_linker,
                                residue_index_offset=512,
                                mask_rate=mask_rate,
                                return_contacts=get_LM_contacts)

            ptm = output["ptm"][0]
            plddt = output["plddt"][0, :, 1].mean()
            plddt_ls.append(float(plddt))
            print(f'{seed} ptm: {ptm:.3f} plddt: {plddt:.1f}')

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return plddt_ls


class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.df = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        sequence = self.df.loc[idx, 'aa_seq']
        name = self.df.loc[idx, 'name']
        return {'name': name, 'sequence': sequence}



def run_seq_sampling_and_save_plddt_with_dataloader(model_dir, input_csv, output_csv, num_recycles=0, chain_linker=2, masking_rate=0.5, get_LM_contacts=True, samples=8, batch_size=1):
    custom_dataset = CustomDataset(input_csv)
    data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=False, num_workers=0)  # Adjust num_workers based on your system

    model = torch.load(model_dir)
    model.cuda().requires_grad_(False)
    model.trunk.set_chunk_size(128)

    batch_count = 0
    download_interval = 300

    for batch_index, batch in enumerate(data_loader):
        sequences = batch['sequence']
        names = batch['name']

        plddt = esmfold_plddt(model, sequences, num_recycles, chain_linker, masking_rate, get_LM_contacts, samples)
        result_df = pd.DataFrame({'name': names, 'sequence': sequences, 'PLDDT': [plddt]})

        if os.path.exists(output_csv):
            result_df.to_csv(output_csv, mode='a', header=False, index=False)
        else:
            result_df.to_csv(output_csv, index=False)

        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        batch_count += 1

        if batch_count % download_interval == 0:
            # Download the CSV file every 500 batches
            download_csv_from_colab(output_csv)

def download_csv_from_colab(csv_filename):
    files.download(csv_filename)



CPU times: user 2.48 s, sys: 1.02 s, total: 3.5 s
Wall time: 2.69 s


In [3]:
run_seq_sampling_and_save_plddt_with_dataloader('esmfold.model', '/content/dmsv2_aa_seq_all_4.csv', '/content/dmsv2_aa_seq_all_4_plddt.csv')

0 ptm: 0.400 plddt: 65.4
1 ptm: 0.403 plddt: 60.9
2 ptm: 0.368 plddt: 59.5
3 ptm: 0.345 plddt: 60.1
4 ptm: 0.520 plddt: 75.1
5 ptm: 0.271 plddt: 56.0
6 ptm: 0.493 plddt: 72.1
7 ptm: 0.549 plddt: 76.3
0 ptm: 0.394 plddt: 64.6
1 ptm: 0.545 plddt: 74.1
2 ptm: 0.386 plddt: 59.2
3 ptm: 0.578 plddt: 75.9
4 ptm: 0.399 plddt: 59.9
5 ptm: 0.609 plddt: 78.2
6 ptm: 0.407 plddt: 64.1
7 ptm: 0.557 plddt: 74.8
0 ptm: 0.390 plddt: 62.1
1 ptm: 0.388 plddt: 60.5
2 ptm: 0.420 plddt: 64.3
3 ptm: 0.366 plddt: 58.4
4 ptm: 0.441 plddt: 70.2
5 ptm: 0.408 plddt: 63.6
6 ptm: 0.424 plddt: 62.1
7 ptm: 0.411 plddt: 62.3
0 ptm: 0.459 plddt: 75.0
1 ptm: 0.429 plddt: 72.2
2 ptm: 0.456 plddt: 72.6
3 ptm: 0.306 plddt: 61.7
4 ptm: 0.456 plddt: 74.3
5 ptm: 0.499 plddt: 76.1
6 ptm: 0.438 plddt: 73.3
7 ptm: 0.459 plddt: 73.8
0 ptm: 0.713 plddt: 85.9
1 ptm: 0.739 plddt: 87.9
2 ptm: 0.734 plddt: 87.1
3 ptm: 0.731 plddt: 87.2
4 ptm: 0.714 plddt: 85.8
5 ptm: 0.715 plddt: 85.9
6 ptm: 0.759 plddt: 89.3
7 ptm: 0.715 plddt: 86.2


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

0 ptm: 0.480 plddt: 64.1
1 ptm: 0.418 plddt: 61.0
2 ptm: 0.616 plddt: 76.1
3 ptm: 0.566 plddt: 69.7
4 ptm: 0.596 plddt: 72.4
5 ptm: 0.471 plddt: 62.7
6 ptm: 0.509 plddt: 67.1
7 ptm: 0.583 plddt: 71.1
0 ptm: 0.476 plddt: 63.1
1 ptm: 0.566 plddt: 71.3
2 ptm: 0.615 plddt: 75.5
3 ptm: 0.563 plddt: 70.5
4 ptm: 0.571 plddt: 72.2
5 ptm: 0.594 plddt: 72.6
6 ptm: 0.511 plddt: 66.4
7 ptm: 0.588 plddt: 73.2
0 ptm: 0.449 plddt: 60.1
1 ptm: 0.437 plddt: 59.7
2 ptm: 0.441 plddt: 59.0
3 ptm: 0.453 plddt: 62.9
4 ptm: 0.521 plddt: 66.0
5 ptm: 0.435 plddt: 57.9
6 ptm: 0.474 plddt: 61.5
7 ptm: 0.536 plddt: 67.3
0 ptm: 0.531 plddt: 70.1
1 ptm: 0.563 plddt: 71.9
2 ptm: 0.527 plddt: 70.4
3 ptm: 0.593 plddt: 75.5
4 ptm: 0.497 plddt: 67.8
5 ptm: 0.611 plddt: 77.3
6 ptm: 0.587 plddt: 79.1
7 ptm: 0.549 plddt: 69.8
0 ptm: 0.673 plddt: 82.0
1 ptm: 0.709 plddt: 84.8
2 ptm: 0.667 plddt: 82.7
3 ptm: 0.612 plddt: 79.1
4 ptm: 0.607 plddt: 80.3
5 ptm: 0.692 plddt: 83.6
6 ptm: 0.606 plddt: 80.5
7 ptm: 0.673 plddt: 82.1


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

0 ptm: 0.686 plddt: 83.5
1 ptm: 0.684 plddt: 82.5
2 ptm: 0.696 plddt: 84.2
3 ptm: 0.663 plddt: 82.0
4 ptm: 0.597 plddt: 77.6
5 ptm: 0.682 plddt: 82.3
6 ptm: 0.667 plddt: 81.3
7 ptm: 0.678 plddt: 81.8
0 ptm: 0.451 plddt: 63.0
1 ptm: 0.497 plddt: 67.9
2 ptm: 0.575 plddt: 72.6
3 ptm: 0.431 plddt: 61.9
4 ptm: 0.588 plddt: 72.9
5 ptm: 0.486 plddt: 67.5
6 ptm: 0.547 plddt: 71.4
7 ptm: 0.434 plddt: 64.1
0 ptm: 0.438 plddt: 63.4
1 ptm: 0.521 plddt: 71.4
2 ptm: 0.474 plddt: 66.3
3 ptm: 0.452 plddt: 65.4
4 ptm: 0.411 plddt: 62.8
5 ptm: 0.483 plddt: 66.6
6 ptm: 0.473 plddt: 65.1
7 ptm: 0.478 plddt: 66.0
0 ptm: 0.424 plddt: 68.5
1 ptm: 0.473 plddt: 73.2
2 ptm: 0.527 plddt: 73.3
3 ptm: 0.458 plddt: 70.5
4 ptm: 0.476 plddt: 71.8
5 ptm: 0.455 plddt: 69.4
6 ptm: 0.433 plddt: 67.5
7 ptm: 0.512 plddt: 74.8
0 ptm: 0.595 plddt: 74.6
1 ptm: 0.601 plddt: 74.6
2 ptm: 0.649 plddt: 78.9
3 ptm: 0.636 plddt: 79.6
4 ptm: 0.693 plddt: 81.9
5 ptm: 0.653 plddt: 79.4
6 ptm: 0.590 plddt: 73.7
7 ptm: 0.585 plddt: 73.5


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

0 ptm: 0.576 plddt: 75.5
1 ptm: 0.600 plddt: 74.3
2 ptm: 0.615 plddt: 75.7
3 ptm: 0.551 plddt: 72.5
4 ptm: 0.510 plddt: 70.6
5 ptm: 0.547 plddt: 70.6
6 ptm: 0.559 plddt: 71.6
7 ptm: 0.531 plddt: 70.1
0 ptm: 0.718 plddt: 83.5
1 ptm: 0.717 plddt: 83.8
2 ptm: 0.745 plddt: 85.8
3 ptm: 0.720 plddt: 84.0
4 ptm: 0.712 plddt: 83.0
5 ptm: 0.724 plddt: 83.8
6 ptm: 0.718 plddt: 83.7
7 ptm: 0.684 plddt: 81.6
0 ptm: 0.514 plddt: 65.1
1 ptm: 0.527 plddt: 65.2
2 ptm: 0.517 plddt: 63.7
3 ptm: 0.559 plddt: 67.3
4 ptm: 0.522 plddt: 65.2
5 ptm: 0.515 plddt: 64.7
6 ptm: 0.566 plddt: 68.9
7 ptm: 0.471 plddt: 60.0
0 ptm: 0.650 plddt: 80.4
1 ptm: 0.608 plddt: 78.0
2 ptm: 0.639 plddt: 79.1
3 ptm: 0.654 plddt: 79.8
4 ptm: 0.616 plddt: 77.6
5 ptm: 0.636 plddt: 78.8
6 ptm: 0.619 plddt: 77.9
7 ptm: 0.632 plddt: 77.9
0 ptm: 0.408 plddt: 61.1
1 ptm: 0.495 plddt: 67.6
2 ptm: 0.308 plddt: 55.2
3 ptm: 0.412 plddt: 64.2
4 ptm: 0.433 plddt: 67.1
5 ptm: 0.422 plddt: 64.6
6 ptm: 0.407 plddt: 66.6
7 ptm: 0.392 plddt: 63.1


In [ ]:
#@title display (optional) {run: "auto"}
#@markdown Note: If samples selected, the model with max pTM is displayed.
import py3Dmol


pymol_color_list = ["#33ff33","#00ffff","#ff33cc","#ffff00","#ff9999","#e5e5e5","#7f7fff","#ff7f00",
                    "#7fff7f","#199999","#ff007f","#ffdd5e","#8c3f99","#b2b2b2","#007fff","#c4b200",
                    "#8cb266","#00bfbf","#b27f7f","#fcd1a5","#ff7f7f","#ffbfdd","#7fffff","#ffff7f",
                    "#00ff7f","#337fcc","#d8337f","#bfff3f","#ff7fff","#d8d8ff","#3fffbf","#b78c4c",
                    "#339933","#66b2b2","#ba8c84","#84bf00","#b24c66","#7f7f7f","#3f3fa5","#a5512b"]

def show_pdb(pdb_str, show_sidechains=False, show_mainchains=False,
             color="pLDDT", chains=None, vmin=50, vmax=90,
             size=(800,480), hbondCutoff=4.0,
             Ls=None,
             animate=False):

  if chains is None:
    chains = 1 if Ls is None else len(Ls)
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js', width=size[0], height=size[1])
  if animate:
    view.addModelsAsFrames(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
  else:
    view.addModel(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
  if color == "pLDDT":
    view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':vmin,'max':vmax}}})
  elif color == "rainbow":
    view.setStyle({'cartoon': {'color':'spectrum'}})
  elif color == "chain":
    for n,chain,color in zip(range(chains),alphabet_list,pymol_color_list):
       view.setStyle({'chain':chain},{'cartoon': {'color':color}})
  if show_sidechains:
    BB = ['C','O','N']
    view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                  {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                  {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                  {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
  if show_mainchains:
    BB = ['C','O','N','CA']
    view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
  view.zoomTo()
  if animate: view.animate()
  return view

color = "confidence" #@param ["confidence", "rainbow", "chain"]
if color == "confidence": color = "pLDDT"
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}
show_pdb(best_pdb_str, color=color,
         show_sidechains=show_sidechains,
         show_mainchains=show_mainchains,
         Ls=lengths).show()

In [ ]:
#@title plot confidence (optional)
#@markdown Note: If samples selected, the model with max pTM is displayed.

dpi = 100 #@param {type:"integer"}

def plot_ticks(Ls):
  Ln = sum(Ls)
  L_prev = 0
  for L_i in Ls[:-1]:
    L = L_prev + L_i
    L_prev += L_i
    plt.plot([0,Ln],[L,L],color="black")
    plt.plot([L,L],[0,Ln],color="black")
  ticks = np.cumsum([0]+Ls)
  ticks = (ticks[1:] + ticks[:-1])/2
  plt.yticks(ticks,alphabet_list[:len(ticks)])

def plot_confidence(output, Ls=None, dpi=100):
  O = parse_output(output)
  if "lm_contacts" in O:
    plt.figure(figsize=(20,4), dpi=dpi)
    plt.subplot(1,4,1)
  else:
    plt.figure(figsize=(15,4), dpi=dpi)
    plt.subplot(1,3,1)

  plt.title('Predicted lDDT')
  plt.plot(O["plddt"])
  if Ls is not None:
    L_prev = 0
    for L_i in Ls[:-1]:
      L = L_prev + L_i
      L_prev += L_i
      plt.plot([L,L],[0,100],color="black")
  plt.xlim(0,O["plddt"].shape[0])
  plt.ylim(0,100)
  plt.ylabel('plDDT')
  plt.xlabel('position')
  plt.subplot(1,4 if "lm_contacts" in O else 3,2)

  plt.title('Predicted Aligned Error')
  Ln = O["pae"].shape[0]
  plt.imshow(O["pae"],cmap="bwr",vmin=0,vmax=30,extent=(0, Ln, Ln, 0))
  if Ls is not None and len(Ls) > 1: plot_ticks(Ls)
  plt.colorbar()
  plt.xlabel('Scored residue')
  plt.ylabel('Aligned residue')

  if "lm_contacts" in O:
    plt.subplot(1,4,3)
    plt.title("contacts from LM")
    plt.imshow(O["lm_contacts"],cmap="Greys",vmin=0,vmax=1,extent=(0, Ln, Ln, 0))
    if Ls is not None and len(Ls) > 1: plot_ticks(Ls)
    plt.subplot(1,4,4)
  else:
    plt.subplot(1,3,3)
  plt.title("contacts from Structure Module")
  plt.imshow(O["sm_contacts"],cmap="Greys",vmin=0,vmax=1,extent=(0, Ln, Ln, 0))
  if Ls is not None and len(Ls) > 1: plot_ticks(Ls)
  return plt

plot_confidence(best_output, Ls=lengths, dpi=dpi)
plt.show()

In [ ]:
#@title download predictions
from google.colab import files
os.system(f"zip {ID}.zip {ID}/*")
files.download(f'{ID}.zip')

In [ ]:
#@title animate outputs (optional)
#@markdown if more than one sample generated
import os
if not os.path.isdir("colabfold"):
  os.system("git clone https://github.com/sokrypton/ColabFold.git")
import ColabFold.beta.colabfold as cf

dpi =  100#@param {type:"integer"}
color_by_plddt = True #@param {type:"boolean"}
use_pca = True
cycle = True

import matplotlib
from matplotlib import animation
from IPython.display import HTML
from sklearn.decomposition import PCA

def mk_animation(xyz, labels, plddt, ref=0, Ls=None, line_w=2.0, dpi=100,color_by_plddt=False):

  def ca_align_to_last(positions, ref):
    def align(P, Q):
      if Ls is None or len(Ls) == 1:
        P_,Q_ = P,Q
      else:
        # align relative to first chain
        P_,Q_ = P[:Ls[0]],Q[:Ls[0]]
      p = P_ - P_.mean(0,keepdims=True)
      q = Q_ - Q_.mean(0,keepdims=True)
      return ((P - P_.mean(0,keepdims=True)) @ cf.kabsch(p,q)) + Q_.mean(0,keepdims=True)

    pos = positions[ref] - positions[ref].mean(0,keepdims=True)
    best_2D_view = pos @ cf.kabsch(pos,pos,return_v=True)

    new_positions = []
    for i in range(len(positions)):
      new_positions.append(align(positions[i],best_2D_view))
    return np.asarray(new_positions)

  # align to reference
  pos = ca_align_to_last(xyz, ref)

  fig, (ax1) = plt.subplots(1)
  fig.set_figwidth(5)
  fig.set_figheight(5)
  fig.set_dpi(dpi)

  xy_min = pos[...,:2].min() - 1
  xy_max = pos[...,:2].max() + 1

  for ax in [ax1]:
    ax.set_xlim(xy_min, xy_max)
    ax.set_ylim(xy_min, xy_max)
    ax.axis(False)

  ims=[]
  for l,pos_,plddt_ in zip(labels,pos,plddt):
    if color_by_plddt:
      img = cf.plot_pseudo_3D(pos_, c=plddt_, cmin=50, cmax=90, line_w=line_w, ax=ax1)
    elif Ls is None or len(Ls) == 1:
      img = cf.plot_pseudo_3D(pos_, ax=ax1, line_w=line_w)
    else:
      c = np.concatenate([[n]*L for n,L in enumerate(Ls)])
      img = cf.plot_pseudo_3D(pos_, c=c, cmap=cf.pymol_cmap, cmin=0, cmax=39, line_w=line_w, ax=ax1)
    ims.append([cf.add_text(f"{l}", ax1),img])

  ani = animation.ArtistAnimation(fig, ims, blit=True, interval=120)
  plt.close()
  return ani.to_html5_video()

labels = np.array([f"seed:{x}" for x in range(len(traj))])
pos = np.array([x["xyz"] for x in traj])
plddt = np.array([x["plddt"] for x in traj])
if use_pca and pos.shape[0] > 1:
  pos_ca = pos
  if lengths is not None and len(lengths) > 1:
    pos_ca = pos_ca[:,:lengths[0]]
  i,j = np.triu_indices(pos_ca.shape[1],k=1)
  pos_ca_dm = np.sqrt(np.square(pos_ca[:,None,:,:] - pos_ca[:,:,None]).sum(-1))[:,i,j]
  pc = PCA(1).fit_transform(pos_ca_dm)[:,0].argsort()
  if cycle:
    pc = np.append(pc,pc[1:-1][::-1])
  pos = pos[pc]
  labels = labels[pc]
  plddt = plddt[pc]
HTML(mk_animation(pos, labels, plddt, Ls=lengths, dpi=dpi, color_by_plddt=color_by_plddt))